In [1]:
import numpy as np
import pandas as pd
from math import sqrt, pi

# Function under the integral
def f(x):
    return np.exp(-x**2 / 2)

# Simpson's rule
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Compute N(t) with convergence log
def compute_N_with_log(t, tol=1e-12):
    a, b = (0, t) if t >= 0 else (t, 0)
    logs = []
    n = 2
    I_old = simpson(f, a, b, n)
    logs.append((n, 0.5 + np.sign(t) * I_old / sqrt(2 * pi)))

    while True:
        n *= 2
        I_new = simpson(f, a, b, n)
        N_val = 0.5 + np.sign(t) * I_new / sqrt(2 * pi)
        logs.append((n, N_val))
        if abs(I_new - I_old) < tol:
            break
        I_old = I_new

    return pd.DataFrame(logs, columns=["n", f"N({t})"])

# Combine results into one DataFrame
df_list = [compute_N_with_log(t) for t in [0.1, 0.5, 1.0]]
merged = df_list[0]
for df in df_list[1:]:
    merged = pd.merge(merged, df, on="n", how="outer")

# Sort by n and format values
merged = merged.sort_values("n").reset_index(drop=True)


In [2]:
merged

,n,N(0.1),N(0.5),N(1.0)
0,2,0.539828,0.691473,0.841529
1,4,0.539828,0.691463,0.841355
2,8,0.539828,0.691463,0.841345
3,16,0.539828,0.691462,0.841345
4,32,0.539828,0.691462,0.841345
5,64,0.539828,0.691462,0.841345
6,128,NaN,0.691462,0.841345
7,256,NaN,0.691462,0.841345
8,512,NaN,0.691462,0.841345
9,1024,NaN,NaN,0.841345


In [4]:
print(merged.to_latex(index=False, float_format="%.13f", escape=False))

\begin{tabular}{rrrr}
\toprule
n & N(0.1) & N(0.5) & N(1.0) \\
\midrule
2 & 0.5398278414043 & 0.6914733395314 & 0.8415290519963 \\
4 & 0.5398278375347 & 0.6914631235012 & 0.8413554878566 \\
8 & 0.5398278372931 & 0.6914625023982 & 0.8413454061391 \\
16 & 0.5398278372780 & 0.6914624638402 & 0.8413447871501 \\
32 & 0.5398278372771 & 0.6914624614343 & 0.8413447486335 \\
64 & 0.5398278372770 & 0.6914624612840 & 0.8413447462288 \\
128 & NaN & 0.6914624612746 & 0.8413447460786 \\
256 & NaN & 0.6914624612741 & 0.8413447460692 \\
512 & NaN & 0.6914624612740 & 0.8413447460686 \\
1024 & NaN & NaN & 0.8413447460685 \\
\bottomrule
\end{tabular}



In [ ]:
import numpy as np
import pandas as pd

# Use user's version of Simpson's rule
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Adaptive Simpson's rule using user's simpson function
def adaptive_simpson(f, a, b, tol):
    results = []
    n = 4
    prev = simpson(f, a, b, n)
    results.append((n, prev))
    while True:
        n *= 2
        curr = simpson(f, a, b, n)
        results.append((n, curr))
        if abs(curr - prev) < tol:
            break
        prev = curr
    return results

def compute_N_t(t, tol=1e-12):
    f = lambda x: np.exp(-x**2 / 2)
    if t == 0:
        return [(4, 0.5)]
    elif t > 0:
        integral_results = adaptive_simpson(f, 0, t, tol)
    else:
        integral_results = adaptive_simpson(f, t, 0, tol)
    return [(n, 0.5 + val / np.sqrt(2 * np.pi)) for n, val in integral_results]

# Compute for t = 0.1, 0.5, 1.0 using the updated simpson
results_01 = compute_N_t(0.1)
results_05 = compute_N_t(0.5)
results_10 = compute_N_t(1.0)

# Merge into DataFrame and align lengths with NaNs where needed
df = pd.DataFrame({
    "n_0.1": pd.Series([v for _, v in results_01], index=[n for n, _ in results_01]),
    "n_0.5": pd.Series([v for _, v in results_05], index=[n for n, _ in results_05]),
    "n_1.0": pd.Series([v for _, v in results_10], index=[n for n, _ in results_10]),
})

print(df.to_latex(float_format="%.13f", index=True, escape=False))

\begin{tabular}{lrrr}
\toprule
 & n_0.1 & n_0.5 & n_1.0 \\
\midrule
4 & 0.5398278375347 & 0.6914631235012 & 0.8413554878566 \\
8 & 0.5398278372931 & 0.6914625023982 & 0.8413454061391 \\
16 & 0.5398278372780 & 0.6914624638402 & 0.8413447871501 \\
32 & 0.5398278372771 & 0.6914624614343 & 0.8413447486335 \\
64 & 0.5398278372770 & 0.6914624612840 & 0.8413447462288 \\
128 & NaN & 0.6914624612746 & 0.8413447460786 \\
256 & NaN & 0.6914624612741 & 0.8413447460692 \\
512 & NaN & 0.6914624612740 & 0.8413447460686 \\
1024 & NaN & NaN & 0.8413447460685 \\
\bottomrule
\end{tabular}



: 

In [1]:
import numpy as np
import pandas as pd

# Updated Simpson's rule (user's version)
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Adaptive Simpson's rule with tolerance checked on N(t), not the integral
def adaptive_simpson_on_N(f, a, b, tol):
    results = []
    n = 4
    prev_integral = simpson(f, a, b, n)
    prev_N = 0.5 + prev_integral / np.sqrt(2 * np.pi)
    results.append((n, prev_N))
    while True:
        n *= 2
        curr_integral = simpson(f, a, b, n)
        curr_N = 0.5 + curr_integral / np.sqrt(2 * np.pi)
        results.append((n, curr_N))
        if abs(curr_N - prev_N) < tol:
            break
        prev_N = curr_N
    return results

def compute_N_t(t, tol=1e-12):
    f = lambda x: np.exp(-x**2 / 2)
    if t == 0:
        return [(4, 0.5)]
    elif t > 0:
        return adaptive_simpson_on_N(f, 0, t, tol)
    else:
        return adaptive_simpson_on_N(f, t, 0, tol)

# Compute for t = 0.1, 0.5, 1.0 with correct tolerance checking
results_01 = compute_N_t(0.1)
results_05 = compute_N_t(0.5)
results_10 = compute_N_t(1.0)

# Combine results into a DataFrame with NaN fill for unequal lengths
df = pd.DataFrame({
    "n_0.1": pd.Series([v for _, v in results_01], index=[n for n, _ in results_01]),
    "n_0.5": pd.Series([v for _, v in results_05], index=[n for n, _ in results_05]),
    "n_1.0": pd.Series([v for _, v in results_10], index=[n for n, _ in results_10]),
})


In [2]:
df

,n_0.1,n_0.5,n_1.0
4,0.539828,0.691463,0.841355
8,0.539828,0.691463,0.841345
16,0.539828,0.691462,0.841345
32,0.539828,0.691462,0.841345
64,NaN,0.691462,0.841345
128,NaN,0.691462,0.841345
256,NaN,0.691462,0.841345
512,NaN,NaN,0.841345


In [5]:
print(df.to_latex(float_format="%.13f", index=True, escape=True))

\begin{tabular}{lrrr}
\toprule
 & n\_0.1 & n\_0.5 & n\_1.0 \\
\midrule
4 & 0.5398278375347 & 0.6914631235012 & 0.8413554878566 \\
8 & 0.5398278372931 & 0.6914625023982 & 0.8413454061391 \\
16 & 0.5398278372780 & 0.6914624638402 & 0.8413447871501 \\
32 & 0.5398278372771 & 0.6914624614343 & 0.8413447486335 \\
64 & NaN & 0.6914624612840 & 0.8413447462288 \\
128 & NaN & 0.6914624612746 & 0.8413447460786 \\
256 & NaN & 0.6914624612741 & 0.8413447460692 \\
512 & NaN & NaN & 0.8413447460686 \\
\bottomrule
\end{tabular}

